## 引言

在日常生活中，大家经常会遇到图像关键信息自动抽取的场景，比如身份证拍照上传自动识别、发票拍照上传自动报销等。


在这个领域，现有的 AI 技术方案已经能解决一部分需求，但是依然存在一些痛点，比如发票的种类样式极其繁多，基于 OCR 文字识别+规则后处理的方案无法有效覆盖全部样式，即泛化性很差。如果要强行覆盖全部样式，成本又太高。


针对这样的问题，开放群岛开源社区大模型SIG联合组长单位之一飞桨团队推出基于文心大模型的全新解决方案——PP-ChatOCR。本项目在飞桨团队的基础上，使用智谱AI的GLM4模型底座，实现使用GLM方案完成自定义信息抽取工作。


本项目使用的数据样例来自互联网，侵权删。

## 1. 检查显卡、显存以及CUDA版本号

In [ ]:
!nvidia-smi

Sat Apr  6 10:35:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

## 2. 安装CUDA版本对应的飞桨框架PaddlePaddle-GPU以及PaddleOCR框架，推荐用飞桨源

In [ ]:
# 安装飞桨框架并验证
!python -m pip install paddlepaddle-gpu==2.6.1.post120 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
import paddle
paddle.utils.run_check()

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [ ]:
# 安装PaddleOCR并验证
!pip install paddleocr
import paddleocr
print(paddleocr.__version__)

2.7.3


## 3. 使用PaddleOCR端到端的解决方案完成文字识别
备注：样例图片来自互联网，侵权删

In [ ]:
from paddleocr import PaddleOCR, draw_ocr

# Paddleocr目前支持的多语言语种可以通过修改lang参数进行切换
# 例如`ch`, `en`, `fr`, `german`, `korean`, `japan`
ocr = PaddleOCR(use_angle_cls=True, lang="ch")  # need to run only once to download and load model into memory
img_path = 'data/sample.jpg'
result = ocr.ocr(img_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)

[2024/04/06 10:45:07] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='/

## 4. 查看识别结果

In [ ]:
print(txts)

['证书号第4119418号', '发明专利证书', '发明名称：一种基于齿轮传动的高精度模具加工装置', 'NIPA', '发明人：蒋伟达', '号：ZL201910522666.6', '专利申请日：2019年06月17日', '专利权人：浙江顺天传动科技股份有限公司', '地', '址：325100浙江省温州市永嘉县乌牛镇东蒙工业园区', '授权公告日：2020年11月27日', '授权公告号：CN110125766B', '国家知识产权局依照中华人民共和国专利法进行审查，决定授子专利权，颁发发明专利', '证书并在专利登记簿上予以登记、专利权自疫权公告之日起生效。专利权期限为二十年，自', '申请日起算。', '专利证书记载专利权登记时的法律状况.专利权的转移、质押、无效、终止、恢复和专', '利权人的姓名或名称、国籍、地址变更等事项记载在专利登记薄上。', '申雨', '局长', '申长雨', '2020年11月27日', '第1页（共2页）', '其他事项参见续页']


## 5. 构造Prompt (Prompt出处来自飞桨团队，针对GLM4进行了一些改动)

In [ ]:
ocr_result=txts
key='{\'证书编号\',\'发明名称\',\'发明人\',\'专利号\',\'专利申请日\',\'专利申请人\',\'地址\'，\'授权公告日\',\'授权公告号\'}'

## 6. 查看构造好的Prompt

In [ ]:
prompt = f"""你现在的任务是从OCR文字识别的结果中提取我指定的关键信息。OCR的文字识别结果使用```符号包围，包含所识别出来的文字，顺序在原始图片中从左至右、从上至下。我指定的关键信息使用[]符号包围。请注意OCR的文字识别结果可能存在长句子换行被切断、不合理的分词、对应错位等问题，你需要结合上下文语义进行综合判断，以抽取准确的关键信息。在返回结果时使用json格式，包含一个key-value对，key值为我指定的关键信息，value值为所抽取的结果。如果认为OCR识别结果中没有关键信息key，则将value赋值为“未找到相关信息”。请只输出json格式的结果，不要包含其它多余文字！下面正式开始：\n\nOCR文字：```{ocr_result}```\n\n要抽取的关键信息：```{key}```。\n\n请只输出json，不要输出额外内容！\n\n你的json输出是："""
print(prompt)

你现在的任务是从OCR文字识别的结果中提取我指定的关键信息。OCR的文字识别结果使用```符号包围，包含所识别出来的文字，顺序在原始图片中从左至右、从上至下。我指定的关键信息使用[]符号包围。请注意OCR的文字识别结果可能存在长句子换行被切断、不合理的分词、对应错位等问题，你需要结合上下文语义进行综合判断，以抽取准确的关键信息。在返回结果时使用json格式，包含一个key-value对，key值为我指定的关键信息，value值为所抽取的结果。如果认为OCR识别结果中没有关键信息key，则将value赋值为“未找到相关信息”。请只输出json格式的结果，不要包含其它多余文字！下面正式开始：

OCR文字：```['证书号第4119418号', '发明专利证书', '发明名称：一种基于齿轮传动的高精度模具加工装置', 'NIPA', '发明人：蒋伟达', '号：ZL201910522666.6', '专利申请日：2019年06月17日', '专利权人：浙江顺天传动科技股份有限公司', '地', '址：325100浙江省温州市永嘉县乌牛镇东蒙工业园区', '授权公告日：2020年11月27日', '授权公告号：CN110125766B', '国家知识产权局依照中华人民共和国专利法进行审查，决定授子专利权，颁发发明专利', '证书并在专利登记簿上予以登记、专利权自疫权公告之日起生效。专利权期限为二十年，自', '申请日起算。', '专利证书记载专利权登记时的法律状况.专利权的转移、质押、无效、终止、恢复和专', '利权人的姓名或名称、国籍、地址变更等事项记载在专利登记薄上。', '申雨', '局长', '申长雨', '2020年11月27日', '第1页（共2页）', '其他事项参见续页']```

要抽取的关键信息：```{'证书编号','发明名称','发明人','专利号','专利申请日','专利申请人','地址'，'授权公告日','授权公告号'}```。

请只输出json，不要输出额外内容！

你的json输出是：


## 7. 使用GLM-4执行信息抽取任务(0 shot)

In [ ]:
!pip install zhipuai
from zhipuai import ZhipuAI
client = ZhipuAI(api_key="") # 填写您自己的APIKey
response = client.chat.completions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=[
        {"role": "user", "content": prompt},
    ],
)
print(response.choices[0].message.content)

```json
{
  "证书编号": "第4119418号",
  "发明名称": "一种基于齿轮传动的高精度模具加工装置",
  "发明人": "蒋伟达",
  "专利号": "ZL201910522666.6",
  "专利申请日": "2019年06月17日",
  "专利申请人": "浙江顺天传动科技股份有限公司",
  "地址": "325100浙江省温州市永嘉县乌牛镇东蒙工业园区",
  "授权公告日": "2020年11月27日",
  "授权公告号": "CN110125766B"
}
```


## 8. 对抽取结果进行清洗

In [ ]:
# 查看结果属性
type(response.choices[0].message.content)

str

In [ ]:
# 使用正则表达式匹配{}之间的内容
import re
pattern = re.compile(r'{.*?}', re.DOTALL)  # 匹配最短的大括号内容，包括换行符
matches = pattern.findall(a)
matches[0]

'{\n  "证书编号": "第4119418号",\n  "发明名称": "一种基于齿轮传动的高精度模具加工装置",\n  "发明人": "蒋伟达",\n  "专利号": "ZL201910522666.6",\n  "专利申请日": "2019年06月17日",\n  "专利申请人": "浙江顺天传动科技股份有限公司",\n  "地址": "325100浙江省温州市永嘉县乌牛镇东蒙工业园区",\n  "授权公告日": "2020年11月27日",\n  "授权公告号": "CN110125766B"\n}'

In [ ]:
import json
json_string=matches[0]
# 使用json.loads()函数解析JSON字符串
parsed_json = json.loads(json_string)
# 打印解析后的JSON对象
print(parsed_json)

{'证书编号': '第4119418号', '发明名称': '一种基于齿轮传动的高精度模具加工装置', '发明人': '蒋伟达', '专利号': 'ZL201910522666.6', '专利申请日': '2019年06月17日', '专利申请人': '浙江顺天传动科技股份有限公司', '地址': '325100浙江省温州市永嘉县乌牛镇东蒙工业园区', '授权公告日': '2020年11月27日', '授权公告号': 'CN110125766B'}


In [ ]:
# 如果您需要以特定的格式访问数据，例如获取“发明名称”
invention_name = parsed_json.get('发明名称')
print(invention_name)

一种基于齿轮传动的高精度模具加工装置
